# Foundations

### Convert to base 2

In [3]:
def convert(x):
    if x == 0: return "0"
    if x == 1: return "1"
    return convert(x//2) + convert(x%2)

convert(34)

'100010'

### Collatz sequence

In [14]:
def collatz(n):
    print(n, " ", end="")
    
    if n == 1: 
        return
    
    if n % 2 == 0: 
        collatz(n // 2)
        return
    
    collatz(3*n + 1)
    
collatz(7)


7  22  11  34  17  52  26  13  40  20  10  5  16  8  4  2  1  

# Classic example

### Subdivisions of a ruller

In [20]:
def ruler(n):
    if n==0: return " "
    return ruler(n-1) + str(n) + ruler(n-1)        
    
ruler(4)

' 1 2 1 3 1 2 1 4 1 2 1 3 1 2 1 '

### Towers of Hanoi

In [32]:
def reverse(direction):
    if direction=="L": return "R"
    if direction=="R": return "L"
    raise Exception("unknown direction: " + direction)

def hanoi(n, direction):
    """
    Assumes that we have one sorted pile of disks with largest disk size n on the middle spindle.
    The returned instruction describes how disks with different sizes must be moved so that the whole sorted pile
    would relocate in the specified direction. Moves are cyclic, L means Left, R means Right. Cyclic means that 
    if we move leftmost to the left it would cycle through to the rightmost position, simialr for other direction.
    """
    if n==0: return " "
    moveSmallerPileAway = hanoi(n-1, reverse(direction))
    moveLargeDisk = str(n) + direction
    moveSmallerPileOnTop = hanoi(n-1, reverse(direction))
    return moveSmallerPileAway + moveLargeDisk + moveSmallerPileOnTop
    

In [34]:
hanoi(4, "L")

' 1R 2L 1R 3R 1R 2L 1R 4L 1R 2L 1R 3R 1R 2L 1R '

Disk 1 always moves right. Disk 2 always moves left. The same for all other disk sizes - they move only in one direction. And if we always move the smallest disk to the right, there is only one possible move available on the other two disks since we can't put anything on top of the smallest disk. This is the algorithm that can be explained to a human. And this algorithm would work without needing lots of stack and that would cover any n.

And the compute complexity is 2\*\*N - 1. For the profecy time estimate we need to check n for 64. And mutiply by the time it takes to move the disk. Let's assume it is one minute.

In [42]:
operations = 2**64 - 1
inSeconds = operations * 60
inCenturies = inSeconds / 60 / 60 / 24 / 365 / 100
inCenturies

350965450413.0432